# SQL GROUP BY: Aggregating and Summarizing Data

## Introduction

**GROUP BY** is one of the most powerful SQL features for data analysis and reporting. It allows you to:
- 📊 **Summarize data** by grouping rows together
- 📈 **Calculate aggregates** like totals, averages, counts
- 🔍 **Analyze patterns** across different categories
- 📋 **Create reports** from detailed transaction data

**Real-world scenarios:**
- "What's the total revenue by product category?"
- "How many orders did each customer place?"
- "What's the average order value by region?"
- "Which products are selling the most?"


**What you'll learn:**
- Understanding GROUP BY fundamentals
- Using aggregate functions (COUNT, SUM, AVG, MIN, MAX)
- GROUP BY with multiple columns
- HAVING clause for filtering groups
- Common patterns and best practices

**Database:** This course uses **Snowflake** database. All examples are Snowflake-compatible. Additional notes for SQL Server are provided where relevant.

**Prerequisites:**
- Understanding of SELECT statements
- Knowledge of WHERE clause
- Basic SQL syntax

---

## Dataset Setup

**Note:** This notebook uses the same dataset created in **"2a SQL Joins.ipynb"**. If you haven't created those tables yet, please run the table creation and data insertion queries from that notebook first.

**Tables we'll use:**
- `customers` - Customer information
- `orders` - Order information  
- `products` - Product catalog
- `order_items` - Items in each order

**Quick Reference - Table Structure:**

**customers:**
- customer_id, first_name, last_name, email, city, country

**orders:**
- order_id, customer_id, order_date, total_amount, status

**products:**
- product_id, product_name, category, price, stock_quantity

**order_items:**
- order_id, product_id, quantity, unit_price

Let's verify the data is available:


In [ ]:
-- Verify tables exist and have data
SELECT 'customers' as table_name, COUNT(*) as row_count FROM customers
UNION ALL
SELECT 'orders', COUNT(*) FROM orders
UNION ALL
SELECT 'products', COUNT(*) FROM products
UNION ALL
SELECT 'order_items', COUNT(*) FROM order_items;


---

## Understanding GROUP BY: The Concept

### What is GROUP BY?

**GROUP BY** groups rows that have the same values in specified columns into summary rows. It's like creating subtotals in a spreadsheet.

**Think of it as:** "Show me one row for each unique combination of these columns, with aggregated values."

### Visual Example

**Before GROUP BY (Raw Data):**
| order_id | customer_id | total_amount |
|----------|-------------|--------------|
| 1001 | 1 | 1029.98 |
| 1002 | 2 | 379.98 |
| 1003 | 1 | 149.99 |
| 1004 | 3 | 1299.97 |
| 1005 | 4 | 89.99 |
| 1006 | 2 | 199.99 |
| 1007 | 5 | 429.98 |

**After GROUP BY customer_id:**
| customer_id | order_count | total_spent |
|-------------|-------------|-------------|
| 1 | 2 | 1179.97 |
| 2 | 2 | 579.97 |
| 3 | 1 | 1299.97 |
| 4 | 1 | 89.99 |
| 5 | 1 | 429.98 |

**What happened:**
- Rows with the same `customer_id` were grouped together
- We calculated aggregates: count of orders and sum of amounts
- Multiple rows became one summary row per customer

### Key Rules

1. **All non-aggregated columns** in SELECT must be in GROUP BY
2. **Aggregate functions** (COUNT, SUM, AVG, etc.) can be used without GROUP BY
3. **WHERE filters rows** before grouping
4. **HAVING filters groups** after grouping

---

## Aggregate Functions

Before diving into GROUP BY, let's understand the aggregate functions we'll use:

### COUNT: Counting Rows

**COUNT(*)** - Counts all rows (including NULLs)  
**COUNT(column)** - Counts non-NULL values in a column  
**COUNT(DISTINCT column)** - Counts unique non-NULL values

### SUM: Adding Values

**SUM(column)** - Sums all values in a column (ignores NULLs)

### AVG: Calculating Averages

**AVG(column)** - Calculates average (ignores NULLs)

### MIN and MAX: Finding Extremes

**MIN(column)** - Finds minimum value (ignores NULLs)  
**MAX(column)** - Finds maximum value (ignores NULLs)

Let's see these in action:


In [ ]:
-- Example: Aggregate functions without GROUP BY (single row result)
SELECT 
    COUNT(*) as total_orders,
    COUNT(DISTINCT customer_id) as unique_customers,
    SUM(total_amount) as total_revenue,
    AVG(total_amount) as avg_order_value,
    MIN(total_amount) as smallest_order,
    MAX(total_amount) as largest_order
FROM orders;


In [ ]:
-- Example 1: Count orders by customer
SELECT 
    customer_id,
    COUNT(*) as order_count
FROM orders
GROUP BY customer_id
ORDER BY order_count DESC;


### Example 2: Total Revenue by Customer

**Question:** What's the total amount each customer has spent?

```sql
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent,
    AVG(total_amount) as avg_order_value
FROM orders
GROUP BY customer_id
ORDER BY total_spent DESC;
```


In [ ]:
-- Example 2: Total revenue by customer
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent,
    AVG(total_amount) as avg_order_value,
    MIN(total_amount) as smallest_order,
    MAX(total_amount) as largest_order
FROM orders
GROUP BY customer_id
ORDER BY total_spent DESC;


### Example 3: Sales by Product Category

**Question:** How much revenue did each product category generate?

**Note:** We need to join `products` with `order_items` to get category information.

```sql
SELECT 
    p.category,
    COUNT(DISTINCT oi.order_id) as number_of_orders,
    SUM(oi.quantity) as total_quantity_sold,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    AVG(oi.unit_price) as avg_unit_price
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY total_revenue DESC;
```


In [ ]:
-- Example 3: Sales by product category
SELECT 
    p.category,
    COUNT(DISTINCT oi.order_id) as number_of_orders,
    SUM(oi.quantity) as total_quantity_sold,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    AVG(oi.unit_price) as avg_unit_price
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY total_revenue DESC;


### Example 4: Orders by Status

**Question:** How many orders are in each status, and what's their total value?

```sql
SELECT 
    status,
    COUNT(*) as order_count,
    SUM(total_amount) as total_value,
    AVG(total_amount) as avg_order_value
FROM orders
GROUP BY status
ORDER BY order_count DESC;
```


In [ ]:
-- Example 4: Orders by status
SELECT 
    status,
    COUNT(*) as order_count,
    SUM(total_amount) as total_value,
    AVG(total_amount) as avg_order_value
FROM orders
GROUP BY status
ORDER BY order_count DESC;


---

## GROUP BY with Multiple Columns

You can group by multiple columns to create more detailed summaries.

### Example 5: Sales by Category and Order Status

**Question:** What's the revenue breakdown by product category and order status?

```sql
SELECT 
    p.category,
    o.status,
    COUNT(*) as order_count,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
INNER JOIN orders o ON oi.order_id = o.order_id
GROUP BY p.category, o.status
ORDER BY p.category, o.status;
```


In [ ]:
-- Example 5: Sales by category and order status
SELECT 
    p.category,
    o.status,
    COUNT(*) as order_count,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
INNER JOIN orders o ON oi.order_id = o.order_id
GROUP BY p.category, o.status
ORDER BY p.category, o.status;


### Example 6: Customer Orders by Country

**Question:** How many orders and total revenue by customer country?

```sql
SELECT 
    c.country,
    COUNT(DISTINCT c.customer_id) as customer_count,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(o.total_amount) as total_revenue,
    AVG(o.total_amount) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country
ORDER BY total_revenue DESC NULLS LAST;
```


In [ ]:
-- Example 6: Customer orders by country
SELECT 
    c.country,
    COUNT(DISTINCT c.customer_id) as customer_count,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(o.total_amount) as total_revenue,
    AVG(o.total_amount) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country
ORDER BY total_revenue DESC NULLS LAST;


---

## HAVING Clause: Filtering Groups

**WHERE** filters rows **before** grouping  
**HAVING** filters groups **after** grouping

### When to Use HAVING

- ✅ Filter based on aggregate values (SUM, COUNT, AVG, etc.)
- ✅ Apply conditions to grouped results
- ❌ Don't use HAVING for non-aggregate conditions (use WHERE instead)

### Example 7: Customers with Multiple Orders

**Question:** Which customers have placed more than 1 order?

```sql
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent
FROM orders
GROUP BY customer_id
HAVING COUNT(*) > 1
ORDER BY order_count DESC;
```


In [ ]:
-- Example 7: Customers with multiple orders (HAVING)
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent
FROM orders
GROUP BY customer_id
HAVING COUNT(*) > 1
ORDER BY order_count DESC;


### Example 8: High-Value Product Categories

**Question:** Which product categories have generated more than $500 in revenue?

```sql
SELECT 
    p.category,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    COUNT(DISTINCT oi.order_id) as order_count
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
HAVING SUM(oi.quantity * oi.unit_price) > 500
ORDER BY total_revenue DESC;
```


In [ ]:
-- Example 8: High-value product categories (HAVING)
SELECT 
    p.category,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    COUNT(DISTINCT oi.order_id) as order_count
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
HAVING SUM(oi.quantity * oi.unit_price) > 500
ORDER BY total_revenue DESC;


### Example 9: WHERE vs HAVING Comparison

**Important:** Use WHERE for row-level filtering, HAVING for group-level filtering.

```sql
-- WHERE: Filter rows BEFORE grouping
-- This filters orders with status 'delivered' first, then groups
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent
FROM orders
WHERE status = 'delivered'  -- Filters rows before grouping
GROUP BY customer_id
HAVING COUNT(*) >= 1  -- Filters groups after grouping
ORDER BY total_spent DESC;
```


In [ ]:
-- Example 9: WHERE vs HAVING comparison
-- WHERE filters rows BEFORE grouping
SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(total_amount) as total_spent
FROM orders
WHERE status = 'delivered'  -- Filter rows first
GROUP BY customer_id
HAVING COUNT(*) >= 1  -- Then filter groups
ORDER BY total_spent DESC;


---

## Common Patterns and Best Practices

### Pattern 1: Including Customer Names in Grouped Results

**Challenge:** GROUP BY uses `customer_id`, but you want to show customer names.

**Solution:** Include the name columns in both SELECT and GROUP BY, or use a subquery/CTE.

```sql
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    COUNT(*) as order_count,
    SUM(o.total_amount) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC NULLS LAST;
```


In [ ]:
-- Pattern 1: Including customer names in grouped results
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    COUNT(*) as order_count,
    SUM(o.total_amount) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC NULLS LAST;


### Pattern 2: Handling NULLs in GROUP BY

**Challenge:** NULL values in grouping columns create separate groups.

**Solution:** Use COALESCE to handle NULLs before grouping.

```sql
SELECT 
    COALESCE(p.category, 'Uncategorized') as category,
    COUNT(*) as product_count,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY COALESCE(p.category, 'Uncategorized')
ORDER BY total_revenue DESC NULLS LAST;
```


In [ ]:
-- Pattern 2: Handling NULLs in GROUP BY
SELECT 
    COALESCE(p.category, 'Uncategorized') as category,
    COUNT(*) as product_count,
    SUM(COALESCE(oi.quantity * oi.unit_price, 0)) as total_revenue
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY COALESCE(p.category, 'Uncategorized')
ORDER BY total_revenue DESC;


### Pattern 3: COUNT DISTINCT vs COUNT

**When to use each:**
- **COUNT(*)** - Count all rows in the group
- **COUNT(column)** - Count non-NULL values
- **COUNT(DISTINCT column)** - Count unique values

```sql
-- Example: Count distinct orders vs total order items
SELECT 
    p.product_id,
    p.product_name,
    COUNT(*) as total_order_items,  -- Total rows (may include duplicates)
    COUNT(DISTINCT oi.order_id) as unique_orders  -- Unique orders
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name
ORDER BY unique_orders DESC;
```


In [ ]:
-- Pattern 3: COUNT DISTINCT vs COUNT
SELECT 
    p.product_id,
    p.product_name,
    COUNT(*) as total_order_items,
    COUNT(DISTINCT oi.order_id) as unique_orders
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name
ORDER BY unique_orders DESC;


### Pattern 4: Calculating Percentages

**Question:** What percentage of total revenue does each category represent?

```sql
SELECT 
    p.category,
    SUM(oi.quantity * oi.unit_price) as category_revenue,
    SUM(oi.quantity * oi.unit_price) * 100.0 / 
        SUM(SUM(oi.quantity * oi.unit_price)) OVER () as revenue_percentage
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY category_revenue DESC;
```

**Note:** This uses a window function (`OVER()`) to calculate the total. We'll cover window functions in detail in another notebook.


In [ ]:
-- Pattern 4: Calculating percentages using window functions
SELECT 
    p.category,
    SUM(oi.quantity * oi.unit_price) as category_revenue,
    ROUND(
        SUM(oi.quantity * oi.unit_price) * 100.0 / 
        SUM(SUM(oi.quantity * oi.unit_price)) OVER (), 
        2
    ) as revenue_percentage
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY category_revenue DESC;


---

## Production-Grade Examples

### Example 10: Customer Lifetime Value Report

**Business Question:** Create a comprehensive customer analysis report.

```sql
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(o.total_amount) as lifetime_value,
    AVG(o.total_amount) as avg_order_value,
    MIN(o.order_date) as first_order_date,
    MAX(o.order_date) as last_order_date,
    DATEDIFF(day, MIN(o.order_date), MAX(o.order_date)) as customer_tenure_days
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.country
ORDER BY lifetime_value DESC NULLS LAST;
```


In [ ]:
-- Example 10: Customer Lifetime Value Report
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(DISTINCT o.order_id) as total_orders,
    COALESCE(SUM(o.total_amount), 0) as lifetime_value,
    COALESCE(AVG(o.total_amount), 0) as avg_order_value,
    MIN(o.order_date) as first_order_date,
    MAX(o.order_date) as last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.country
ORDER BY lifetime_value DESC NULLS LAST;


### Example 11: Product Performance Dashboard

**Business Question:** Generate a product performance report for inventory management.

```sql
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    p.price as current_price,
    p.stock_quantity,
    COUNT(DISTINCT oi.order_id) as times_ordered,
    SUM(oi.quantity) as total_units_sold,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    AVG(oi.unit_price) as avg_selling_price,
    CASE 
        WHEN SUM(oi.quantity) = 0 THEN 'Not Sold'
        WHEN SUM(oi.quantity) < 5 THEN 'Slow Moving'
        ELSE 'Fast Moving'
    END as sales_velocity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name, p.category, p.price, p.stock_quantity
ORDER BY total_revenue DESC NULLS LAST;
```


In [ ]:
-- Example 11: Product Performance Dashboard
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    p.price as current_price,
    p.stock_quantity,
    COUNT(DISTINCT oi.order_id) as times_ordered,
    COALESCE(SUM(oi.quantity), 0) as total_units_sold,
    COALESCE(SUM(oi.quantity * oi.unit_price), 0) as total_revenue,
    COALESCE(AVG(oi.unit_price), p.price) as avg_selling_price,
    CASE 
        WHEN COALESCE(SUM(oi.quantity), 0) = 0 THEN 'Not Sold'
        WHEN COALESCE(SUM(oi.quantity), 0) < 5 THEN 'Slow Moving'
        ELSE 'Fast Moving'
    END as sales_velocity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name, p.category, p.price, p.stock_quantity
ORDER BY total_revenue DESC NULLS LAST;


### Example 12: Monthly Sales Report

**Business Question:** Show sales trends by month.

```sql
SELECT 
    EXTRACT(YEAR FROM o.order_date) as order_year,
    EXTRACT(MONTH FROM o.order_date) as order_month,
    COUNT(DISTINCT o.order_id) as order_count,
    COUNT(DISTINCT o.customer_id) as unique_customers,
    SUM(o.total_amount) as total_revenue,
    AVG(o.total_amount) as avg_order_value
FROM orders o
GROUP BY EXTRACT(YEAR FROM o.order_date), EXTRACT(MONTH FROM o.order_date)
ORDER BY order_year, order_month;
```


In [ ]:
-- Example 12: Monthly Sales Report
SELECT 
    EXTRACT(YEAR FROM o.order_date) as order_year,
    EXTRACT(MONTH FROM o.order_date) as order_month,
    COUNT(DISTINCT o.order_id) as order_count,
    COUNT(DISTINCT o.customer_id) as unique_customers,
    SUM(o.total_amount) as total_revenue,
    AVG(o.total_amount) as avg_order_value
FROM orders o
GROUP BY EXTRACT(YEAR FROM o.order_date), EXTRACT(MONTH FROM o.order_date)
ORDER BY order_year, order_month;


---

## Common Mistakes and How to Avoid Them

### Mistake 1: Forgetting Non-Aggregated Columns in GROUP BY

**Error:** "Column 'column_name' must appear in the GROUP BY clause"

**Wrong:**
```sql
SELECT customer_id, first_name, COUNT(*)
FROM customers
GROUP BY customer_id;  -- Missing first_name in GROUP BY
```

**Correct:**
```sql
SELECT customer_id, first_name, COUNT(*)
FROM customers
GROUP BY customer_id, first_name;  -- Include all non-aggregated columns
```

### Mistake 2: Using WHERE Instead of HAVING

**Error:** Trying to filter on aggregate values with WHERE

**Wrong:**
```sql
SELECT customer_id, COUNT(*) as order_count
FROM orders
WHERE COUNT(*) > 1  -- ERROR: Can't use aggregate in WHERE
GROUP BY customer_id;
```

**Correct:**
```sql
SELECT customer_id, COUNT(*) as order_count
FROM orders
GROUP BY customer_id
HAVING COUNT(*) > 1;  -- Use HAVING for aggregate conditions
```

### Mistake 3: Not Handling NULLs

**Problem:** NULL values create separate groups, which may not be desired.

**Solution:** Use COALESCE before grouping.

### Mistake 4: Incorrect COUNT Usage

**Common Confusion:**
- `COUNT(*)` - Counts all rows (including NULLs)
- `COUNT(column)` - Counts non-NULL values only
- `COUNT(DISTINCT column)` - Counts unique non-NULL values

Choose the right one based on your needs!

---

## Best Practices Summary

### ✅ DO's

1. **Include all non-aggregated columns in GROUP BY**
2. **Use WHERE for row filtering, HAVING for group filtering**
3. **Handle NULLs with COALESCE before grouping**
4. **Use COUNT(DISTINCT) when you need unique counts**
5. **Use meaningful column aliases for aggregates**
6. **Order results logically (usually by aggregate value)**
7. **Use LEFT JOIN when you want all records from the left table**
8. **Document complex GROUP BY logic**

### ❌ DON'Ts

1. **Don't forget columns in GROUP BY**
2. **Don't use aggregate functions in WHERE clause**
3. **Don't ignore NULL handling**
4. **Don't use COUNT(*) when you mean COUNT(column)**
5. **Don't group by columns you don't need**
6. **Don't forget to handle NULLs in calculations**

---

## Practice Problems

Now it's time to test your understanding! Try to solve these problems on your own before looking at the solutions.


## Problem 1: Top 3 Customers by Revenue

**Problem Statement:**
Find the top 3 customers by total revenue. Show:
- Customer ID and full name
- Total number of orders
- Total amount spent
- Average order value

Order by total amount spent (descending).


In [ ]:
-- Write your solution here for Problem 1


### Solution to Problem 1

**Approach:**
1. Join customers with orders
2. Group by customer
3. Calculate aggregates
4. Order by total spent descending
5. Limit to top 3

```sql
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    COUNT(*) as total_orders,
    SUM(o.total_amount) as total_spent,
    AVG(o.total_amount) as avg_order_value
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC
LIMIT 3;
```

**Explanation:**
- `INNER JOIN` ensures we only get customers with orders
- `GROUP BY` includes customer_id and name columns
- `COUNT(*)` counts orders per customer
- `SUM()` calculates total revenue
- `AVG()` calculates average order value
- `LIMIT 3` restricts to top 3 results


## Problem 2: Products Never Sold

**Problem Statement:**
Find all products that have never been ordered. Show:
- Product ID and name
- Category
- Current price
- Stock quantity

Order by product name.


In [ ]:
-- Write your solution here for Problem 2


### Solution to Problem 2

**Approach:**
1. Use LEFT JOIN to include all products
2. Group by product
3. Use HAVING to filter products with no orders

```sql
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    p.price,
    p.stock_quantity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name, p.category, p.price, p.stock_quantity
HAVING COUNT(oi.order_id) = 0
ORDER BY p.product_name;
```

**Alternative Solution (without GROUP BY):**
```sql
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    p.price,
    p.stock_quantity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
WHERE oi.order_id IS NULL
ORDER BY p.product_name;
```

**Explanation:**
- `LEFT JOIN` ensures all products are included
- `GROUP BY` groups by product attributes
- `HAVING COUNT(oi.order_id) = 0` filters products with no orders
- Alternative uses `WHERE oi.order_id IS NULL` (simpler, no GROUP BY needed)


## Problem 3: Category Revenue Analysis

**Problem Statement:**
Show revenue analysis by product category:
- Category name
- Number of different products in category
- Total quantity sold
- Total revenue
- Average unit price

Only include categories with revenue greater than $200.
Order by total revenue (descending).


In [ ]:
-- Write your solution here for Problem 3


### Solution to Problem 3

**Approach:**
1. Join products with order_items
2. Group by category
3. Calculate aggregates
4. Use HAVING to filter categories with revenue > $200

```sql
SELECT 
    p.category,
    COUNT(DISTINCT p.product_id) as product_count,
    SUM(oi.quantity) as total_quantity_sold,
    SUM(oi.quantity * oi.unit_price) as total_revenue,
    AVG(oi.unit_price) as avg_unit_price
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
HAVING SUM(oi.quantity * oi.unit_price) > 200
ORDER BY total_revenue DESC;
```

**Explanation:**
- `INNER JOIN` gets products that have been ordered
- `COUNT(DISTINCT p.product_id)` counts unique products per category
- `SUM(oi.quantity)` calculates total units sold
- `SUM(oi.quantity * oi.unit_price)` calculates total revenue
- `HAVING` filters groups with revenue > $200
- `ORDER BY` sorts by revenue descending


## Problem 4: Order Status Summary

**Problem Statement:**
Create a summary of orders by status:
- Order status
- Count of orders
- Total revenue
- Average order value
- Minimum order value
- Maximum order value

Order by order count (descending).


In [ ]:
-- Write your solution here for Problem 4


### Solution to Problem 4

**Approach:**
1. Group by status
2. Calculate multiple aggregates
3. Order by order count

```sql
SELECT 
    status,
    COUNT(*) as order_count,
    SUM(total_amount) as total_revenue,
    AVG(total_amount) as avg_order_value,
    MIN(total_amount) as min_order_value,
    MAX(total_amount) as max_order_value
FROM orders
GROUP BY status
ORDER BY order_count DESC;
```

**Explanation:**
- Simple GROUP BY on status column
- Multiple aggregate functions show different metrics
- `COUNT(*)` counts orders per status
- `SUM()`, `AVG()`, `MIN()`, `MAX()` provide revenue statistics
- `ORDER BY` sorts by order count


## Problem 5: Customer Orders by Country

**Problem Statement:**
Show order statistics by customer country:
- Country name
- Number of customers
- Number of orders
- Total revenue
- Average order value

Include all countries, even if they have no orders (show 0 for metrics).
Order by total revenue (descending, NULLs last).


In [ ]:
-- Write your solution here for Problem 5


### Solution to Problem 5

**Approach:**
1. Use LEFT JOIN to include all countries
2. Group by country
3. Use COALESCE to handle NULLs
4. Order with NULLS LAST

```sql
SELECT 
    c.country,
    COUNT(DISTINCT c.customer_id) as customer_count,
    COUNT(DISTINCT o.order_id) as order_count,
    COALESCE(SUM(o.total_amount), 0) as total_revenue,
    COALESCE(AVG(o.total_amount), 0) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country
ORDER BY total_revenue DESC NULLS LAST;
```

**Explanation:**
- `LEFT JOIN` ensures all countries are included
- `COUNT(DISTINCT)` counts unique customers and orders
- `COALESCE()` handles NULLs from countries with no orders
- `NULLS LAST` puts countries with no orders at the end


---

## Summary

**Key Takeaways:**

1. **GROUP BY groups rows** with the same values into summary rows
2. **Aggregate functions** (COUNT, SUM, AVG, MIN, MAX) calculate values for each group
3. **All non-aggregated columns** must be in GROUP BY clause
4. **WHERE filters rows** before grouping, **HAVING filters groups** after grouping
5. **NULL values** create separate groups - handle them with COALESCE
6. **COUNT DISTINCT** counts unique values, COUNT counts all rows
7. **LEFT JOIN** preserves all records from the left table in grouped results

**Common Use Cases:**
- ✅ Summarizing sales by category, region, or time period
- ✅ Calculating customer lifetime value
- ✅ Analyzing product performance
- ✅ Creating reports and dashboards
- ✅ Finding top/bottom performers
- ✅ Identifying patterns in data


---

**End of Notebook**
